In [1]:
# default_exp datahub

# DataHub

> Do Stuff.

In [2]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import datahub.emitter.mce_builder as builder
from datahub_provider.operators.datahub import DatahubEmitterOperator
from netdata_airflow_utils.utils import dest
from typing import List


In [3]:
#export


def make_datahub_lineage_op(upstream_urns: List[str], downstream_urn: str, task_id: str = None,
                            datahub_conn_id: str = 'datahub_rest_default', env: str = 'DEV',
                            source_type: str = 'bigquery') -> DatahubEmitterOperator:
    """Aims to simplify the creation of a Datahub lineage by returning an initialized `DatahubEmitterOperator`.
    """
    if task_id is None:
        task_id = f"lineage__{'.'.join(downstream_urn.split('.')[-2:])}"
    return DatahubEmitterOperator(
        task_id=task_id,
        datahub_conn_id=datahub_conn_id,
        mces=[
            builder.make_lineage_mce(
                upstream_urns=[builder.make_dataset_urn(source_type, dest(urn), env=env) for urn in upstream_urns],
                downstream_urn=builder.make_dataset_urn(source_type, dest(downstream_urn), env=env),
            )
        ],
    )


In [ ]:
#tests
#hide
dh_op = make_datahub_lineage_op(['p.d.t1'], 'p.d.t2', task_id='foo')
assert type(dh_op) == DatahubEmitterOperator
assert dh_op.task_id == 'foo'

